In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
def get_input(name):
    with open(f'{name}.txt') as f:
        return f.read().split('\n')

In [3]:
import numpy as np
import sys
sys.path.insert(0, '../')
from aoc_utils import get_grid_neighbours

# Part 1

In [47]:
nei_dict = {'n': ['nw', 'n', 'ne'],
            'e': ['ne', 'e', 'se'],
            's': ['se', 's', 'sw'],
            'w': ['sw', 'w', 'nw']}

def get_elves_in_neighbour_tiles(elf_field, neis_8):
    elves_in_tiles = dict()
    for direction, tile in neis_8.items():
        try:
            elves_found = elf_field[tile]
        except KeyError:
            elves_found = []
        elves_in_tiles[direction] = elves_found    
        
    return elves_in_tiles

def check_neighbour_tiles_for_elves(direction, elves_in_tiles):
    return any([any(elves_in_tiles[d]) for d in nei_dict[direction]])

    
class elf:
    def __init__(self, location):
        self.location = location
        self.suggested_tile = None
        
    def __repr__(self):
        return f'Elf at {self.location}, suggested: {self.suggested_tile}'
    
    def get_tile_suggestion(self, elf_field):
        neis_8 = get_grid_neighbours(self.location, 8, as_dict=True)
        
        elves_in_tiles = get_elves_in_neighbour_tiles(elf_field, neis_8)
        if not any(elves_in_tiles.values()):
            return None
        else:
            for direction in order_of_movements:
                if not check_neighbour_tiles_for_elves(direction, elves_in_tiles):
                    return neis_8[direction]
        return None

    def suggest_tile(self, elf_field):
        self.suggested_tile = self.get_tile_suggestion(elf_field)
        
    def approve_suggestion(self):
        if self.suggested_tile is not None:
            self.location = self.suggested_tile
            self.suggested_tile = None

        
def update_elf_field(elves):
    elf_field = dict()
    for e in elves:
        if not e.location in elf_field:
            elf_field[e.location] = [e]
        else:
            elf_field[e.location].append(e)
    return elf_field

def update_tentative_elf_field(elves):
    tentative_elf_field = dict()
    for e in elves:
        if (tile := e.suggested_tile) is not None:
            if not tile in tentative_elf_field:
                tentative_elf_field[tile] = [e]
            else:
                tentative_elf_field[tile].append(e)
    return tentative_elf_field


def initialize_elves(elf_arrangement):
    elves = []
    for i in range(len(elf_arrangement)):
        for j in range(len(elf_arrangement[0])):
            if elf_arrangement[i][j] == '#':
                elves.append(elf((i, j)))
    return elves

In [62]:
elf_arrangement = get_input('input')
elves = initialize_elves(elf_arrangement)
elf_field = update_elf_field(elves)

order_of_movements = np.array(['n', 's', 'w', 'e'])

for _ in range(10):
    for e in elves:
        e.suggest_tile(elf_field)
    tentative_elf_field = update_tentative_elf_field(elves)
    for suggested_tile, elves_that_want_to_move_here in tentative_elf_field.items():
        if len(elves_that_want_to_move_here) == 1:
            elves_that_want_to_move_here[0].approve_suggestion()
    
    elf_field = update_elf_field(elves)
    order_of_movements = np.roll(order_of_movements, -1)

In [63]:
ymin = min(elf_field.keys(), key=lambda t: t[0])[0]
ymax = max(elf_field.keys(), key=lambda t: t[0])[0]

xmin = min(elf_field.keys(), key=lambda t: t[1])[1]
xmax = max(elf_field.keys(), key=lambda t: t[1])[1]

area = (xmax - xmin + 1)*(ymax - ymin + 1)
print(area - len(elves))

4070


# Part 2

In [67]:
elf_arrangement = get_input('input')
elves = initialize_elves(elf_arrangement)
elf_field = update_elf_field(elves)

order_of_movements = np.array(['n', 's', 'w', 'e'])
finished = False
rnd = 0
while not finished:
    rnd += 1
    for e in elves:
        e.suggest_tile(elf_field)
    tentative_elf_field = update_tentative_elf_field(elves)
    if not tentative_elf_field:
        finished = True
        
    for suggested_tile, elves_that_want_to_move_here in tentative_elf_field.items():
        if len(elves_that_want_to_move_here) == 1:
            elves_that_want_to_move_here[0].approve_suggestion()
    
    elf_field = update_elf_field(elves)
    order_of_movements = np.roll(order_of_movements, -1)

In [68]:
print(rnd)

881
